In [1]:
import psycopg2 as ps
import warnings 
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
import datetime

## Подключение к базе данных и сбор датасетов

Подключение

In [2]:
conn = ps.connect(host="46.34.136.91", port = 25432, database="postgres", 
                  user="aaa_student", password="student_@@@_2022")

Сбор датасетов. User_info, Item_info выгружаем полностью

In [3]:
user_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.services_verification_experiment_segment", conn)

In [4]:
user_info.head()

,user_id,experiment_group,UserType,RegistrationTime
0,386931286,30_discount,Company,2013-04-29 16:49:41.611735
1,386932096,no_discount,Company,2013-04-29 17:57:23.113672
2,386943455,60_discount,Private,2013-04-30 17:40:49.198317
3,386948456,30_discount,Private,2013-05-01 10:47:50.612032
4,386958876,60_discount,Private,2013-05-02 14:41:52.244914


In [5]:
verification_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.services_verification_discounts", conn)

In [6]:
verification_info.head()

,user_id,item_id,discount_day,experiment
0,49000737,11352751363,2021-10-01,1_verification_test
1,4255271,78048500109,2021-10-10,1_verification_test
2,111000153,101676750179,2021-10-02,1_verification_test
3,110750728,162344000369,2021-10-14,1_verification_test
4,143655000006,252307750608,2021-10-09,1_verification_test


Обращаем внимание, что скидку могут дать на несколько объявлений

In [7]:
verification_info.shape

(29963, 4)

In [8]:
verification_info['user_id'].unique().shape

(22658,)

In [12]:
verification_info.to_csv('verification_info.csv')

In [9]:
verification_info['discount_day'] = pd.to_datetime(verification_info['discount_day'], format='%Y-%m-%d')

Далее, сджойним таблицу с payments и таблицу с информацией по объявлениям. Ограничим таблицу item_vertical = 1, transaction_type = 3.

In [10]:
users_item_and_payment_info = sqlio.read_sql_query("SELECT * FROM PUBLIC.user_payments_info AS p LEFT JOIN PUBLIC.user_item_info As i ON p.user_id = i.user_id AND p.item_id = i.item_id WHERE item_vertical = '1' AND transaction_type = '3'", conn)

In [46]:
users_item_and_payment_info.to_csv('users_item_and_payment_info.csv')

Да, я, кстати, записал на всякий случай себе данные в csv таблицы) мало ли, потом соединение сломается)

In [11]:
users_item_and_payment_info.head()

,user_id,item_id,event_time,transaction_type,transaction_subtype,amount_net,item_id,user_id,item_vertical,item_region,item_creation_time
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,3,16,30.000000,1099385250423,290613250103,1,75,2021-08-02 16:17:26.063798
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,3,17,42.307692,1072235500005,879029000009,1,46,2021-06-21 06:41:56.826799
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,3,18,26.846154,1082999250118,253395500010,1,31,2021-07-05 12:48:24.340016
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,3,19,68.538462,1074243500089,865041000002,1,42,2021-06-22 20:17:26.295496
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,3,10,104.692308,1074243500089,865041000002,1,42,2021-06-22 20:17:26.295496


In [12]:
users_item_and_payment_info.shape

(906985, 11)

Небольшой прикол: доставая данные таким образом - получаются столбцы с одинаковыми именами) и возникают трудности с удалением дубликата (даже по номеру столбца). Пришлось написать функцию, которая переименовывает столбцы.

In [13]:
def rename_pandas_duplicated_columnes(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique(): 
        cols[cols[cols == dup].index.values.tolist()] = [dup + '.' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    return df

In [14]:
df = rename_pandas_duplicated_columnes(users_item_and_payment_info)

In [15]:
df.head()

,user_id,item_id,event_time,transaction_type,transaction_subtype,amount_net,item_id.1,user_id.1,item_vertical,item_region,item_creation_time
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,3,16,30.000000,1099385250423,290613250103,1,75,2021-08-02 16:17:26.063798
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,3,17,42.307692,1072235500005,879029000009,1,46,2021-06-21 06:41:56.826799
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,3,18,26.846154,1082999250118,253395500010,1,31,2021-07-05 12:48:24.340016
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,3,19,68.538462,1074243500089,865041000002,1,42,2021-06-22 20:17:26.295496
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,3,10,104.692308,1074243500089,865041000002,1,42,2021-06-22 20:17:26.295496


О, теперь разные )

In [16]:
df = df[df.columns[[0, 1, 2, 4, 5, 9, 10]]]

In [17]:
df.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,16,30.000000,75,2021-08-02 16:17:26.063798
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,17,42.307692,46,2021-06-21 06:41:56.826799
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,18,26.846154,31,2021-07-05 12:48:24.340016
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,19,68.538462,42,2021-06-22 20:17:26.295496
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,10,104.692308,42,2021-06-22 20:17:26.295496


Ну и, наверное, будет проще сразу сджойнить с информацией по юзерам и таблицей с верификацией.

In [18]:
df1 = df.merge(user_info, on='user_id', how='left')

In [19]:
df1.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,16,30.000000,75,2021-08-02 16:17:26.063798,60_discount,Private,2017-02-12 10:26:05.467496
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,17,42.307692,46,2021-06-21 06:41:56.826799,60_discount,Private,2021-06-21 06:21:00.208680
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,18,26.846154,31,2021-07-05 12:48:24.340016,no_discount,Private,2016-09-25 04:38:05.875127
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,19,68.538462,42,2021-06-22 20:17:26.295496,30_discount,Private,2021-05-27 18:50:16.175609
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,10,104.692308,42,2021-06-22 20:17:26.295496,30_discount,Private,2021-05-27 18:50:16.175609


Сразу доберем в датасет информацию с верификациями, чтобы потом не возвращаться.

In [20]:
data = df1.merge(verification_info, on=['user_id', 'item_id'], how='left')

In [21]:
data.head()

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime,discount_day,experiment
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,16,30.000000,75,2021-08-02 16:17:26.063798,60_discount,Private,2017-02-12 10:26:05.467496,NaT,NaN
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,17,42.307692,46,2021-06-21 06:41:56.826799,60_discount,Private,2021-06-21 06:21:00.208680,2021-09-23,1_verification_test
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,18,26.846154,31,2021-07-05 12:48:24.340016,no_discount,Private,2016-09-25 04:38:05.875127,NaT,NaN
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,19,68.538462,42,2021-06-22 20:17:26.295496,30_discount,Private,2021-05-27 18:50:16.175609,NaT,NaN
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,10,104.692308,42,2021-06-22 20:17:26.295496,30_discount,Private,2021-05-27 18:50:16.175609,NaT,NaN


Худо-бедно датасет собран. Идем считать метрики.

## Подсчет item_per_users, ARPU в группах до начала эксперимента

### item_per_users

Первым делом посчитаем item_per_user. Нас интересуют объявления, созданные до начала эксперимента: 2021-09-15, фильтровать будем item_creation_time этой датой.

In [22]:
df_for_item_per_users = data[['user_id', 'item_id', 'item_creation_time', 'experiment_group']]

In [23]:
df_for_item_per_users.head()

,user_id,item_id,item_creation_time,experiment_group
0,290613250103,1.099385e+12,2021-08-02 16:17:26.063798,60_discount
1,879029000009,1.072236e+12,2021-06-21 06:41:56.826799,60_discount
2,253395500010,1.082999e+12,2021-07-05 12:48:24.340016,no_discount
3,865041000002,1.074244e+12,2021-06-22 20:17:26.295496,30_discount
4,865041000002,1.074244e+12,2021-06-22 20:17:26.295496,30_discount


In [24]:
df_for_item_per_users['item_creation_time'] = pd.to_datetime(df_for_item_per_users['item_creation_time'])

In [25]:
df_for_item_per_users_before_test = df_for_item_per_users[df_for_item_per_users['item_creation_time'] < '2021-09-15']

In [26]:
item_counts = df_for_item_per_users_before_test.groupby(['experiment_group', 'user_id'])[['item_id']].count()

In [27]:
item_counts.head()

item_id
experiment_group user_id         
30_discount      789            1
                 2893           3
                 3449           2
                 3789           2
                 4137           5

Небольшой тест: действительно у юзера 2893 создано три объявления до 15 сентября

In [28]:
data.loc[(data['experiment_group'] == '30_discount') & (data['user_id'] == 2893)]

,user_id,item_id,event_time,transaction_subtype,amount_net,item_region,item_creation_time,experiment_group,UserType,RegistrationTime,discount_day,experiment
182820,2893,1.165118e+12,2021-11-24 09:56:45,10,101.538462,25,2021-11-24 09:56:48.784227,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
375810,2893,8.759838e+11,2021-08-24 06:45:39,10,101.538462,25,2020-08-25 10:37:17.954182,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
433891,2893,8.912750e+11,2021-09-06 13:17:12,10,50.846154,25,2020-09-20 23:32:12.640702,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN
530030,2893,7.824090e+11,2021-08-01 12:47:52,10,50.846154,25,2020-01-21 22:44:31.264369,30_discount,Private,2010-12-24 08:48:26.573836,NaT,NaN


Найдем среднее количество items на одного юзера в каждой группе:

In [29]:
mean_item_per_user = item_counts.groupby('experiment_group').mean()

In [30]:
mean_item_per_user

,item_id
experiment_group,
30_discount,6.352235
60_discount,6.590310
no_discount,6.162525


### ARPU

Для подсчета ARPU до теста будем ограничивать event_time - время покупки

In [31]:
df_for_ARPU = data[['user_id', 'item_id', 'event_time', 'experiment_group', 'amount_net']]

In [32]:
df_for_ARPU.head()

,user_id,item_id,event_time,experiment_group,amount_net
0,290613250103,1.099385e+12,2021-08-02 16:20:33.000000,60_discount,30.000000
1,879029000009,1.072236e+12,2021-09-22 08:33:16.000000,60_discount,42.307692
2,253395500010,1.082999e+12,2021-07-21 22:40:56.000000,no_discount,26.846154
3,865041000002,1.074244e+12,2021-06-24 16:01:00.000000,30_discount,68.538462
4,865041000002,1.074244e+12,2021-06-24 09:37:28.401409,30_discount,104.692308


Аналогично, ограничим время:

In [33]:
df_for_ARPU_before_test = df_for_ARPU[df_for_ARPU['event_time'] < '2021-09-15']

Посчитаем суммы платежей по юзерам:

In [34]:
user_payments = df_for_ARPU_before_test.groupby(['experiment_group', 'user_id'])[['amount_net']].sum()

In [35]:
user_payments.head()

amount_net
experiment_group user_id            
30_discount      789       37.692308
                 2893     203.230769
                 3449      69.307692
                 3789      60.153846
                 4137      40.461538

Для юзера 2893 в тесте выше сумма сходится )

Посчитаем ARPU

In [36]:
ARPU = user_payments.groupby('experiment_group').mean()

In [37]:
ARPU

,amount_net
experiment_group,
30_discount,230.898654
60_discount,239.768872
no_discount,226.872500


### Выводы

Чтобы сделать выводы о схожести групп, применим тест на равенство средних попарно

$$H_0: item-per-user_{test} = item-per-user_{control}$$
$$H_1: item-per-user_{test} \neq item-per-user_{control}$$

То же самое и с ARPU:

$$H_0: ARPU_{test} = ARPU_{control}$$
$$H_1: ARPU_{test} \neq ARPU_{control}$$

Ожидаем принятия нулевых гипотез

Начнем с item_per_user

In [62]:
item_counts = item_counts.reset_index()

Создаем группы для сравнения

In [63]:
groups = [('no_discount', '30_discount'), ('no_discount', '60_discount'), ('30_discount', '60_discount')] 

In [64]:
for group in groups:
    
    # делим выборки на тест и контроль в зависимости от группы
    test = item_counts[(item_counts['experiment_group'] == group[0])]
    control = item_counts[(item_counts['experiment_group'] == group[1])]
    
    # применяем двувыборочный t-test без альтернативы (берем p-value)
    pvalue = stats.ttest_ind(test['item_id'], control['item_id'])[1]
    
    # считаем разницу средних
    difference_of_means = test['item_id'].mean() - control['item_id'].mean()
    
    # выводим результат
    print(f'Пара:{group}, Разница средних:{round(difference_of_means, 4)}, Pvalue:{round(pvalue, 4)}')

Пара:('no_discount', '30_discount'), Разница средних:-0.1897, Pvalue:0.4138
Пара:('no_discount', '60_discount'), Разница средних:-0.4278, Pvalue:0.2581
Пара:('30_discount', '60_discount'), Разница средних:-0.2381, Pvalue:0.5352


Можно сделать вывод о сбалансированноси групп по метрике item_per_user

Следующий ARPU

In [66]:
user_payments = user_payments.reset_index()

In [67]:
for group in groups:
    
    # делим выборки на тест и контроль в зависимости от группы
    test = user_payments[(user_payments['experiment_group'] == group[0])]
    control = user_payments[(user_payments['experiment_group'] == group[1])]
    
    # применяем двувыборочный t-test без альтернативы (берем p-value)
    pvalue = stats.ttest_ind(test['amount_net'], control['amount_net'])[1]
    
    # считаем разницу средних
    difference_of_means = test['amount_net'].mean() - control['amount_net'].mean()
    
    # выводим результат
    print(f'Пара:{group}, Разница средних ARPU:{round(difference_of_means, 4)}, Pvalue:{round(pvalue, 4)}')

Пара:('no_discount', '30_discount'), Разница средних ARPU:-4.0262, Pvalue:0.6391
Пара:('no_discount', '60_discount'), Разница средних ARPU:-12.8964, Pvalue:0.2994
Пара:('30_discount', '60_discount'), Разница средних ARPU:-8.8702, Pvalue:0.4721


Также делаем вывод о сбалансированности по ARPU

## Попарные сравнения средних с помощью t-test

На первом шаге решил поступить чрезмерно примитивно: просто попарно сравнить средние с помощью t-test, чтобы получить хоть какие-то результаты.

Но сперва нужно получить экспериментальные выборки.

Теперь отберем данные позже 2021-09-15

### Формализация гипотез

Перед проведением тестирования, наверное, было бы неплохо наметить свои ожидания. 

 

В конечном счете, мы ожидаем увеличение числа объявлений на одного юзера. Скидка должна простимулировать генерить пользователей больше объявлений

$$H_0: item-per-user_{0} = item-per-user_{30}$$
$$H_1: item-per-user_{0} < item-per-user_{30}$$

Ожидаем непринятия нулевой гипотезы.


Более того, можно предположить, что бОльшая скидка сподвигнет приобретать услуги сильнее, что даст более высокий стимул генерить больше объявлений в дальнейшем.


$$H_0: item-per-user_{30} = item-per-user_{60}$$
$$H_1: item-per-user_{30} < item-per-user_{60}$$

И в конечном счете:

$$H_0: item-per-user_{0} = item-per-user_{60}$$
$$H_1: item-per-user_{0} < item-per-user_{60}$$


Везде ожидаем непринятия нулевых гипотез

То же самое и с ARPU. Большее число объявлений даст больше шансов для приобретения услуг для каких-либо из них

$$H_0: ARPU_0 = ARPU_{30}$$
$$H_1: ARPU_0 < ARPU_{30}$$

Та же логика и для бОльшей скидки:

$$H_0: ARPU_{30}= ARPU_{60}$$
$$H_1: ARPU_{30} < ARPU_{60}$$

И в конечном счете:

$$H_0: ARPU_{0} = ARPU_{60}$$
$$H_1: ARPU_{0} < ARPU_{60}$$

Ожидаем непринятия нулевых гипотез

Замечание.

Выбор односторонней и двухсторонней гипотезы - очень тонкая тема. На этот счет есть разные мнения и даже на разных курсах Авито лекторы придерживались разных взглядов. В данном случае, я решил поставить односторонние, исходя из бизнес-логики, подразумевая, что просадка поставленных метрик маловероятна с введением скидок.

В то же время, при проверке на схожесть групп, гипотезы ставились двухсторонние, поскольку мы не делаем никаких предположений о потенциальном изменении, потенциальное отклонение там может быть равновероятно как в ту, так и в другую сторону.

### item_per_users

In [52]:
df_for_item_per_users_exp = df_for_item_per_users[df_for_item_per_users['item_creation_time'] >= '2021-09-15']

In [53]:
df_for_item_per_users_exp.head()

,user_id,item_id,item_creation_time,experiment_group
414,393200750029,1.124603e+12,2021-09-15 22:23:50.302226,no_discount
461,6551250341,1.104105e+12,2021-11-03 13:46:57.273453,60_discount
770,913080500018,1.114767e+12,2021-09-26 10:32:15.484792,30_discount
771,913080500018,1.114767e+12,2021-09-26 10:32:15.484792,30_discount
772,913080500018,1.114767e+12,2021-09-26 10:32:15.484792,30_discount


In [54]:
item_counts_exp = df_for_item_per_users_exp.groupby(['experiment_group', 'user_id'])[['item_id']].count()

In [55]:
item_counts_exp = item_counts_exp.reset_index()

In [56]:
item_counts_exp.head()

,experiment_group,user_id,item_id
0,30_discount,2893,1
1,30_discount,5487,5
2,30_discount,16966,1
3,30_discount,22983,1
4,30_discount,269071,1


P.S. Да, можно было загнать код с t-test в одну функцию и сделать все красиво, но я слишком торопился(( поправлю попозже

Создаем попарные группы

In [57]:
groups = [('no_discount', '30_discount'), ('no_discount', '60_discount'), ('30_discount', '60_discount')] 

In [45]:
from scipy import stats

In [60]:
# запускаем цикл по каждой паре
for group in groups:
    
    # делим выборки на тест и контроль в зависимости от группы
    test = item_counts_exp[(item_counts_exp['experiment_group'] == group[0])]
    control = item_counts_exp[(item_counts_exp['experiment_group'] == group[1])]
    
    # применяем двувыборочный t-test с альтернативой (берем p-value)
    pvalue = stats.ttest_ind(test['item_id'], control['item_id'], alternative='less')[1]
    
    # считаем разницу средних
    difference_of_means = test['item_id'].mean() - control['item_id'].mean()
    
    # выводим результат
    print(f'Пара:{group}, Разница средних:{round(difference_of_means, 4)}, Pvalue:{round(pvalue, 4)}')

Пара:('no_discount', '30_discount'), Разница средних:0.274, Pvalue:0.7437
Пара:('no_discount', '60_discount'), Разница средних:-2.0002, Pvalue:0.1955
Пара:('30_discount', '60_discount'), Разница средних:-2.2742, Pvalue:0.1608


Видим, что в ростом скидки до 60 процентов средние количества объявлений на юзера увеличиваются, но эти изменения не являются статистически значимыми. 

### ARPU

Аналогично, ограничиваем данные временем эксперимента

In [51]:
df_for_ARPU_exp = df_for_ARPU[df_for_ARPU['event_time'] >= '2021-09-15']

In [69]:
user_payments_exp = df_for_ARPU_exp.groupby(['experiment_group', 'user_id'])[['amount_net']].sum()

In [70]:
user_payments_exp = user_payments_exp.reset_index()

In [71]:
for group in groups:
    
    # делим выборки на тест и контроль в зависимости от группы
    test = user_payments_exp[(user_payments_exp['experiment_group'] == group[0])]
    control = user_payments_exp[(user_payments_exp['experiment_group'] == group[1])]
    
    # применяем двувыборочный t-test с альтернативой (берем p-value)
    pvalue = stats.ttest_ind(test['amount_net'], control['amount_net'], alternative='less')[1]
    
    # считаем разницу средних
    difference_of_means = test['amount_net'].mean() - control['amount_net'].mean()
    
    # выводим результат
    print(f'Пара:{group}, Разница средних ARPU:{round(difference_of_means, 4)}, Pvalue:{round(pvalue, 4)}')

Пара:('no_discount', '30_discount'), Разница средних ARPU:-1.2411, Pvalue:0.4495
Пара:('no_discount', '60_discount'), Разница средних ARPU:-2.959, Pvalue:0.4033
Пара:('30_discount', '60_discount'), Разница средних ARPU:-1.7179, Pvalue:0.4383


Ну что могу сказать) как-то грустно) пока нет никаких статистических различий.. ушел перепроверяться)